In [4]:
import requests
from lxml import etree

In [5]:
WIKI_PATH = "cswiki-latest-pages-articles.xml"

In [6]:
def get_page_by_title(xml_path, search_title):
    context = etree.iterparse(xml_path, events=('end',), tag='{*}page')
    for _, elem in context:
        title = elem.findtext('{*}title')
        if title == search_title:
            text_elem = elem.find('.//{*}revision/{*}text')
            raw_text = text_elem.text if text_elem is not None else ""
            return title, raw_text.strip()
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    return None, None

In [7]:
def wiki_search_fallback(query: str, lang="cs", top_k=3):
    url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    response = requests.get(url, params=params).json()
    results = response.get("query", {}).get("search", [])
    return [r["title"] for r in results[:top_k]]

In [16]:
titles = wiki_search_fallback("Jakým průmyslem se proslavilo město Nový Bor?")
print(titles)

['Střední uměleckoprůmyslová škola sklářská Kamenický Šenov', 'René Roubíček', 'Polsko']


In [17]:
title, content = get_page_by_title(WIKI_PATH, titles[0])
if content:
    print(f"Title: {title}\n\n{content}...")
else:
    print("Page not found.")

Title: Střední uměleckoprůmyslová škola sklářská Kamenický Šenov

{{Infobox - škola
  | logo = Secondary glass making school Kamenický Šenov.jpg
  | název = Střední uměleckoprůmyslová škola sklářská Kamenický Šenov
  | zkratka = SUPŠS Kamenický Šenov
  | ředitel = Doc. MgA. Pavel Kopřiva, PhD.
  | zástupce = Mgr. Jakub Jelínek
  | počet žáků = <!--- počet žáků -->
  | datum založení = [[1856]]
  | obory = &nbsp;
82-41-M/13 Výtvarné zpracování skla a světelných objektů
<br />
82-41-M/01 Užitá malba
  | izo = 
}}
'''Střední uměleckoprůmyslová škola sklářská Kamenický Šenov''' (ve zkratce SUPŠS Kamenický Šenov) je nejstarší sklářskou školou ve [[Střední Evropa|střední Evropě]].<ref>{{Citace monografie|příjmení = LANGHAMER|jméno = Antonín|příjmení2 = |jméno2 = |titul = The legend of Bohemian glass: a thousand years of glassmaking in the heart of Europe|vydání = 1st English ed.|vydavatel = Tigris|místo = Zlín|rok = 2003|počet stran = 294|strany = 242|isbn = 80-86062-11-2|url = https://books